# ROS Manipulation in 5 Days

## Unit 5: Grasping

<p style="background:green;color:white;">**SUMMARY**</p>

Estimated time of completion: <b>2h</b><br><br>
This unit will show you how to perform motion planning with Python. By completing this unit, you will be able to create a Python program that performs motion planning on your robot.

<p style="background:green;color:white;">**END OF SUMMARY**</p>

## What is Grasping?

When working with manipulator robots, one of the main goals to reach is picking an object up from one position, and placing it in another one, which is commonly known as pick & place. We call <b>Grasping</b> the process of picking an object up by the robot end-effector (which can be a hand, a gripper, etc.). Although this may sound like a very simple task, it is not. In fact, it's a very complex process, because lots of variables need to be taken into account when picking the object.
<br><br>
We humans handle our grasping using our intelligence, but in a robot, we have to create rules for it. One of the constraints in grasping is force; the gripper/end-effector should adjust the grasping force for picking up the object, but not make any deformation on the object while grasping it.

## Grasping using MoveIt!

First of all, we are going to see how to perform Grasping in MoveIt. This means, without controlling the real robot.

And for that, we are going to be using the <i><b>moveit_simple_grasps</b></i> package. This package is very useful in order to generate grasp poses for simple objects, such as blocks or cylinders. And it's quite simple, which helps in the process of learning. This package takes the position of the object to be grasped as input, and generates the necessary grasping sequences in order to pick the object up. 

This package already supports robots such as Baxter, REEM, and Clam arm, among others. But it is quite easy to interface any other custom manipulator robot to the package, without having to modify too many of the codes. 

<p style="background:#EE9023;color:white;">**Exercise 6.1**</p>
<br>
a) First of all, you will create a new package called <i><b>myrobot_grasping</b></i>. Inside this package, create a new folder named <i><b>launch</b></i>, which will contain a launch file called <i><b>grasp_generator_server.launch</b></i>. Copy the following content inside this file.

In [ ]:
<launch>
  <arg name="robot" default="fetch"/>
  <arg name="group"        default="arm"/>
  <arg name="end_effector" default="gripper"/>

  <node pkg="moveit_simple_grasps" type="moveit_simple_grasps_server" name="moveit_simple_grasps_server">
    <param name="group"        value="$(arg group)"/> 
    <param name="end_effector" value="$(arg end_effector)"/>
    <rosparam command="load" file="$(find myrobot_grasping)/config/$(arg robot)_grasp_data.yaml"/> 
  </node>

</launch>

Basically, what we are doing in this launch file is starting a grasp server that will provide grasp sequences to a grasp client node. We need to specify the following into the node:

* <b>Planning Group of the arm</b>
* <b>Planning Group of end-effector</b>

Also, we are loading the <i><b>fetch_grasp_data.yaml</b></i> file, which will contain detailed information about the gripper. We are going to create this file right now.

b) Inside this package, create a new folder named <i><b>config</b></i>, and create a new file named <i><b>fetch_grasp_data.yaml</b></i> inside this folder. Copy the following contents into this file.

In [ ]:
base_link: 'base_link'
gripper:
  #The end effector name for grasping
  end_effector_name: 'gripper'
  
  # Gripper joints
  joints: ['l_gripper_finger_joint', 'r_gripper_finger_joint']
  
  #Posture of grippers before grasping
  pregrasp_posture: [0.048, 0.048]
  pregrasp_time_from_start: 4.0
  grasp_posture: [0.016, 0.016]
  grasp_time_from_start: 4.0
  postplace_time_from_start: 4.0
  
  # Desired pose from end effector to grasp [x, y, z] + [R, P, Y]
  grasp_pose_to_eef: [-0.12, 0.0, 0.0]
  grasp_pose_to_eef_rotation: [0.0, 0.0, 0.0]
  end_effector_parent_link: 'wrist_roll_link'

So, in this file, we are basically providing detailed information about the gripper. These parameters can be tuned in order to improve the grasping process. The most important parameters that we are defining here are the following:

* <b>end_effector_name</b>: The name of the end-effector group, as stated in the MoveIt package.
* <b>joints</b>: The joints that form the gripper.
* <b>pregrasp_posture</b>: The position of the gripper before grasping (OPEN position).
* <b>pregrasp_time_from_start</b>: The time to wait before grasping.
* <b>grasp_posture</b>: The position of the gripper when grasping (CLOSED position).
* <b>grasp_time_from_start</b>: The time to wait after grasping.
* <b>postplace_time_from_start</b>: The name of the end-effector.
* <b>grasp_pose_to_eef</b>: The desired pose from end-effector to grasp - [x,y,z].
* <b>grasp_pose_to_eef</b>: The desired pose from end-effector to grasp - [roll, pitch, yaw].
* <b>end_effector_parent_link</b>: The name of the link that connects the gripper to the robot.

<p style="background:#EE9023;color:white;">**End of Exercise 6.1**</p>

So, you have now created the basic structure to create a Grasping Server, which will allow you to send object positions to this server, and it will provide you with a sequence in order to grasp the specified object.
<br><br>
But, for that, we need to communicate with that server, right? And that's exactly what you are going to do in the next section!

## Creating a pick and place task

There are various ways in which we can perform pick and place tasks. For instance, we could send the robot a predefined sequence of joint values directly, which will cause the robot to always perform the same predefined motion. So, for this case, we must always place the object in the same position. This method is called <b>forward kinematics</b> because you need to previously know the sequence of joint values that are required in order to perform a certain trajectory.
<br><br>
Another method would be by using <b>inverse kinematics</b> (IK), without any vision feedback. In this case, we provide the robot with the Pose (x, y, and z) where the object to picked is, and by doing some IK calculations, the robot would know which motions it needs to perform in order to reach the object's Pose.
<br><br>
Finally, another method would be to use <b>inverse kinematics</b>, but with vision support or feedback. In this case, we would use a node that would identify the Pose of the object by reading the data of the sensors of the robot; for instance, a Kinect camera. Then, this vision node would provide the Pose of the object, and again, by using IK calculations, the robot would know the required motions to perform in order to reach the object.
<br><br>
You are probably thinking that the most efficient and stylish method is the third one, and you're right. But, this method would require you to have some Object Recognition knowledge, which is not the subject of this course. So, for this course, we are going to use the second method, which in the end, is the same as the third one, without the object recognition part. If you already have some Perception knowledge, you can freely add an Object Recognition node that sends the Pose of the object, without having to set this Pose in the code itself.

So, to summarize, what you are going to do in this section of the course is create a node that will send an object Pose to the Grasping Server, in order to receive the appropiate joint values to reach that object. Well then, enough talking, let's get to work!

<p style="background:#EE9023;color:white;">**Exercise 6.2**</p>

a) Inside the package that you created in the previous exercise, create a new folder named <i><b>scripts</b></i>. Inside this folder, create a new file named <i><b>pick_and_place.py</b></i>. Copy the following code inside it.

In [ ]:
#!/usr/bin/env python

import rospy

from moveit_commander import RobotCommander, PlanningSceneInterface
from moveit_commander import roscpp_initialize, roscpp_shutdown

from actionlib import SimpleActionClient, GoalStatus

from geometry_msgs.msg import Pose, PoseStamped, PoseArray, Quaternion
from moveit_msgs.msg import PickupAction, PickupGoal
from moveit_msgs.msg import PlaceAction, PlaceGoal
from moveit_msgs.msg import PlaceLocation
from moveit_msgs.msg import MoveItErrorCodes
from moveit_simple_grasps.msg import GenerateGraspsAction, GenerateGraspsGoal, GraspGeneratorOptions

from tf.transformations import quaternion_from_euler

import sys
import copy
import numpy


# Create dict with human readable MoveIt! error codes:
moveit_error_dict = {}
for name in MoveItErrorCodes.__dict__.keys():
    if not name[:1] == '_':
        code = MoveItErrorCodes.__dict__[name]
        moveit_error_dict[code] = name


class Pick_Place:
    def __init__(self):
        # Retrieve params:
        self._table_object_name = rospy.get_param('~table_object_name', 'Grasp_Table')
        self._grasp_object_name = rospy.get_param('~grasp_object_name', 'Grasp_Object')

        self._grasp_object_width = rospy.get_param('~grasp_object_width', 0.01)

        self._arm_group     = rospy.get_param('~arm', 'arm')
        self._gripper_group = rospy.get_param('~gripper', 'gripper')

        self._approach_retreat_desired_dist = rospy.get_param('~approach_retreat_desired_dist', 0.2)
        self._approach_retreat_min_dist = rospy.get_param('~approach_retreat_min_dist', 0.1)

        # Create (debugging) publishers:
        self._grasps_pub = rospy.Publisher('grasps', PoseArray, queue_size=1, latch=True)
        self._places_pub = rospy.Publisher('places', PoseArray, queue_size=1, latch=True)

        # Create planning scene and robot commander:
        self._scene = PlanningSceneInterface()
        self._robot = RobotCommander()

        rospy.sleep(1.0)

        # Clean the scene:
        self._scene.remove_world_object(self._table_object_name)
        self._scene.remove_world_object(self._grasp_object_name)

        # Add table and Coke can objects to the planning scene:
        self._pose_table    = self._add_table(self._table_object_name)
        self._pose_coke_can = self._add_grasp_block_(self._grasp_object_name)

        rospy.sleep(1.0)

        # Define target place pose:
        self._pose_place = Pose()

        self._pose_place.position.x = self._pose_coke_can.position.x 
        self._pose_place.position.y = self._pose_coke_can.position.y - 0.10
        self._pose_place.position.z = self._pose_coke_can.position.z + 0.08

        self._pose_place.orientation = Quaternion(*quaternion_from_euler(0.0, 0.0, 0.0))

        # Retrieve groups (arm and gripper):
        self._arm     = self._robot.get_group(self._arm_group)
        self._gripper = self._robot.get_group(self._gripper_group)

        # Create grasp generator 'generate' action client:
        self._grasps_ac = SimpleActionClient('/moveit_simple_grasps_server/generate', GenerateGraspsAction)
        if not self._grasps_ac.wait_for_server(rospy.Duration(5.0)):
            rospy.logerr('Grasp generator action client not available!')
            rospy.signal_shutdown('Grasp generator action client not available!')
            return

        # Create move group 'pickup' action client:
        self._pickup_ac = SimpleActionClient('/pickup', PickupAction)
        if not self._pickup_ac.wait_for_server(rospy.Duration(5.0)):
            rospy.logerr('Pick up action client not available!')
            rospy.signal_shutdown('Pick up action client not available!')
            return

        # Create move group 'place' action client:
        self._place_ac = SimpleActionClient('/place', PlaceAction)
        if not self._place_ac.wait_for_server(rospy.Duration(5.0)):
            rospy.logerr('Place action client not available!')
            rospy.signal_shutdown('Place action client not available!')
            return

        # Pick Coke can object:
        while not self._pickup(self._arm_group, self._grasp_object_name, self._grasp_object_width):
            rospy.logwarn('Pick up failed! Retrying ...')
            rospy.sleep(1.0)

        rospy.loginfo('Pick up successfully')

        # Place Coke can object on another place on the support surface (table):
        while not self._place(self._arm_group, self._grasp_object_name, self._pose_place):
            rospy.logwarn('Place failed! Retrying ...')
            rospy.sleep(1.0)

        rospy.loginfo('Place successfully')

    def __del__(self):
        # Clean the scene:
        self._scene.remove_world_object(self._grasp_object_name)
        self._scene.remove_world_object(self._table_object_name)

    def _add_table(self, name):
        p = PoseStamped()
        p.header.frame_id = self._robot.get_planning_frame()
        p.header.stamp = rospy.Time.now()

        p.pose.position.x = 1.0
        p.pose.position.y = 0.08
        p.pose.position.z = 1.02

        q = quaternion_from_euler(0.0, 0.0, numpy.deg2rad(90.0))
        p.pose.orientation = Quaternion(*q)

        # Table size from ~/.gazebo/models/table/model.sdf, using the values
        # for the surface link.
        self._scene.add_box(name, p, (0.86, 0.86, 0.02))

        return p.pose

    def _add_grasp_block_(self, name):
        p = PoseStamped()
        p.header.frame_id = self._robot.get_planning_frame()
        p.header.stamp = rospy.Time.now()

        p.pose.position.x = 0.62   
        p.pose.position.y = 0.21
        p.pose.position.z = 1.07

        q = quaternion_from_euler(0.0, 0.0, 0.0)
        p.pose.orientation = Quaternion(*q)

        # Coke can size from ~/.gazebo/models/coke_can/meshes/coke_can.dae,
        # using the measure tape tool from meshlab.
        # The box is the bounding box of the coke cylinder.
        # The values are taken from the cylinder base diameter and height.
        self._scene.add_box(name, p, (0.077, 0.077, 0.070))

        return p.pose

    def _generate_grasps(self, pose, width):
        """
        Generate grasps by using the grasp generator generate action; based on
        server_test.py example on moveit_simple_grasps pkg.
        """

        # Create goal:
        goal = GenerateGraspsGoal()

        goal.pose  = pose
        goal.width = width

        options = GraspGeneratorOptions()
        # simple_graps.cpp doesn't implement GRASP_AXIS_Z!
        #options.grasp_axis      = GraspGeneratorOptions.GRASP_AXIS_Z
        options.grasp_direction = GraspGeneratorOptions.GRASP_DIRECTION_UP
        options.grasp_rotation  = GraspGeneratorOptions.GRASP_ROTATION_FULL

        # @todo disabled because it works better with the default options
        #goal.options.append(options)

        # Send goal and wait for result:
        state = self._grasps_ac.send_goal_and_wait(goal)
        if state != GoalStatus.SUCCEEDED:
            rospy.logerr('Grasp goal failed!: %s' % self._grasps_ac.get_goal_status_text())
            return None

        grasps = self._grasps_ac.get_result().grasps

        # Publish grasps (for debugging/visualization purposes):
        self._publish_grasps(grasps)

        return grasps

    def _generate_places(self, target):
        """
        Generate places (place locations), based on
        https://github.com/davetcoleman/baxter_cpp/blob/hydro-devel/
        baxter_pick_place/src/block_pick_place.cpp
        """

        # Generate places:
        places = []
        now = rospy.Time.now()
        for angle in numpy.arange(0.0, numpy.deg2rad(360.0), numpy.deg2rad(1.0)):
            # Create place location:
            place = PlaceLocation()

            place.place_pose.header.stamp = now
            place.place_pose.header.frame_id = self._robot.get_planning_frame()

            # Set target position:
            place.place_pose.pose = copy.deepcopy(target)

            # Generate orientation (wrt Z axis):
            q = quaternion_from_euler(0.0, 0.0, angle )
            place.place_pose.pose.orientation = Quaternion(*q)

            # Generate pre place approach:
            place.pre_place_approach.desired_distance = self._approach_retreat_desired_dist
            place.pre_place_approach.min_distance = self._approach_retreat_min_dist

            place.pre_place_approach.direction.header.stamp = now
            place.pre_place_approach.direction.header.frame_id = self._robot.get_planning_frame()

            place.pre_place_approach.direction.vector.x =  0
            place.pre_place_approach.direction.vector.y =  0
            place.pre_place_approach.direction.vector.z = 0.2

            # Generate post place approach:
            place.post_place_retreat.direction.header.stamp = now
            place.post_place_retreat.direction.header.frame_id = self._robot.get_planning_frame()

            place.post_place_retreat.desired_distance = self._approach_retreat_desired_dist
            place.post_place_retreat.min_distance = self._approach_retreat_min_dist

            place.post_place_retreat.direction.vector.x = 0
            place.post_place_retreat.direction.vector.y = 0
            place.post_place_retreat.direction.vector.z = 0.2

            # Add place:
            places.append(place)

        # Publish places (for debugging/visualization purposes):
        self._publish_places(places)

        return places

    def _create_pickup_goal(self, group, target, grasps):
        """
        Create a MoveIt! PickupGoal
        """

        # Create goal:
        goal = PickupGoal()

        goal.group_name  = group
        goal.target_name = target

        goal.possible_grasps.extend(grasps)

        goal.allowed_touch_objects.append(target)

        goal.support_surface_name = self._table_object_name

        # Configure goal planning options:
        goal.allowed_planning_time = 7.0

        goal.planning_options.planning_scene_diff.is_diff = True
        goal.planning_options.planning_scene_diff.robot_state.is_diff = True
        goal.planning_options.plan_only = False
        goal.planning_options.replan = True
        goal.planning_options.replan_attempts = 20

        return goal

    def _create_place_goal(self, group, target, places):
        """
        Create a MoveIt! PlaceGoal
        """

        # Create goal:
        goal = PlaceGoal()

        goal.group_name           = group
        goal.attached_object_name = target

        goal.place_locations.extend(places)

        # Configure goal planning options:
        goal.allowed_planning_time = 7.0

        goal.planning_options.planning_scene_diff.is_diff = True
        goal.planning_options.planning_scene_diff.robot_state.is_diff = True
        goal.planning_options.plan_only = False
        goal.planning_options.replan = True
        goal.planning_options.replan_attempts = 20

        return goal

    def _pickup(self, group, target, width):
        """
        Pick up a target using the planning group
        """

        # Obtain possible grasps from the grasp generator server:
        grasps = self._generate_grasps(self._pose_coke_can, width)

        # Create and send Pickup goal:
        goal = self._create_pickup_goal(group, target, grasps)

        state = self._pickup_ac.send_goal_and_wait(goal)
        if state != GoalStatus.SUCCEEDED:
            rospy.logerr('Pick up goal failed!: %s' % self._pickup_ac.get_goal_status_text())
            return None

        result = self._pickup_ac.get_result()

        # Check for error:
        err = result.error_code.val
        if err != MoveItErrorCodes.SUCCESS:
            rospy.logwarn('Group %s cannot pick up target %s!: %s' % (group, target, str(moveit_error_dict[err])))

            return False

        return True

    def _place(self, group, target, place):
        """
        Place a target using the planning group
        """

        # Obtain possible places:
        places = self._generate_places(place)

        # Create and send Place goal:
        goal = self._create_place_goal(group, target, places)

        state = self._place_ac.send_goal_and_wait(goal)
        if state != GoalStatus.SUCCEEDED:
            rospy.logerr('Place goal failed!: %s' % self._place_ac.get_goal_status_text())
            return None

        result = self._place_ac.get_result()

        # Check for error:
        err = result.error_code.val
        if err != MoveItErrorCodes.SUCCESS:
            rospy.logwarn('Group %s cannot place target %s!: %s' % (group, target, str(moveit_error_dict[err])))

            return False

        return True

    def _publish_grasps(self, grasps):
        """
        Publish grasps as poses, using a PoseArray message
        """

        if self._grasps_pub.get_num_connections() > 0:
            msg = PoseArray()
            msg.header.frame_id = self._robot.get_planning_frame()
            msg.header.stamp = rospy.Time.now()

            for grasp in grasps:
                p = grasp.grasp_pose.pose

                msg.poses.append(Pose(p.position, p.orientation))

            self._grasps_pub.publish(msg)

    def _publish_places(self, places):
        """
        Publish places as poses, using a PoseArray message
        """

        if self._places_pub.get_num_connections() > 0:
            msg = PoseArray()
            msg.header.frame_id = self._robot.get_planning_frame()
            msg.header.stamp = rospy.Time.now()

            for place in places:
                msg.poses.append(place.place_pose.pose)

            self._places_pub.publish(msg)


def main():
    p = Pick_Place()

    rospy.spin()

if __name__ == '__main__':
    roscpp_initialize(sys.argv)
    rospy.init_node('pick_and_place')

    main()

    roscpp_shutdown()

Yes, I know... this is a lot of code! And there's no explanation at all! But, don't worry if you don't understand much of the code at this point. After completing this exercise, we are going to go into detail about the main parts of the code. Trust me!

b) First of all, launch the demo.launch file generated in the MoveIt package in Chapter 3.
<br>
<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">Execute in WebShell #1</p>
</th>
</tr>
</table>

In [ ]:
roslaunch fetch_moveit_config demo.launch

c) Set the "Planning Attempts" option to something like 6.

<img src="img/planning_att.png" width="300" />

d) Next, launch the grasping server you created in the previous exercise.
<br>
<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">Execute in WebShell #1</p>
</th>
</tr>
</table>

In [ ]:
roslaunch myrobot_grasping grasp_generator_server.launch

e) Finally, launch this Python script that you've just created, and move to the MoveIt RViz screen in order to visualize what's going on.
<br>
<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">Execute in WebShell #1</p>
</th>
</tr>
</table>

In [ ]:
rosrun myrobot_grasping pick_and_place.py

<p style="color:red;"><b>IMPORTANT READ:</b> Since the update of the system to ROS Kinetic, the simulation of the pick and place action in MoveIt may fail. If you try to run it and it fails, just move one and continue following the Notebook. You will do this pick and place action in the next exercise, which is exactly the same as this one but executing it in the Gazebo simulated robot. That exercise should be working fine. Sorry for the inconveniences.</p>

<p style="background:#AE0202;color:white;">**Expected Result for Exercise 6.2**</p>

Initial status:

<img src="img/grasp_demo1.png" width="600" />

Going to grasp pose:

<img src="img/grasp_demo2.png" width="600" />

Grasping object:

<img src="img/grasp_demo3.png" width="600" />

<p style="background:#EE9023;color:white;">**End of Exercise 6.2**</p>

Several things happened in the previous exercise, so let's go by parts!

* First of all, you saw a couple of green blocks appearing in the RViz screen, right? Can you guess what they are?
<br>
Well, that were the table and the object to be grasped, of course! Inside the pick_and_place.py script, we have created these 2 objects and added them into the MoveIt Planning Scene. Whenever you add an object to the Planning Scene, it will appear in this way.
* Second, the <b>Pick</b> Action starts. After getting the grasp object position, our node sends this position to the grasp server, which will generate IK and check if there's any valid IK in order to pick the object up. If it finds any feasible IK solution, the arm will begin to execute the specified motions in order to pick the object up.</li>
* After the object is picked by the gripper, the <b>Place</b> Action starts. Again, our node will send a Pose to the grasp server where the arm should place the object. Then, the server will check for a valid IK solution for that specified Pose. If it finds any valid solution, the gripper will move to that position and release the object.</li>

You can have a look at the topics <b>/grasp</b> and <b>/place</b> in order to better see what's going on.
<br><br>
Ok, so now you have a better idea of what happened in the previous exercise. Let's take a deeper look into the code in order to see how all of these work.

### Explaining the code

First of all, let's check how we created and added both the table and the grasping object.

In [ ]:
def _add_table(self, name):
    p = PoseStamped()
    p.header.frame_id = self._robot.get_planning_frame()
    p.header.stamp = rospy.Time.now()
    #Table position
    p.pose.position.x = 0.45
    p.pose.position.y = 0.0
    p.pose.position.z = 0.22
    q = quaternion_from_euler(0.0, 0.0, numpy.deg2rad(90.0))
    p.pose.orientation = Quaternion(*q)
    # Table size 
    self._scene.add_box(name, p, (0.5, 0.4, 0.02))
    return p.pose

In this section of the code, what we are doing is creating and adding a table to the Planning Scene. Quite self explanatory, right?
<br><br>
We first create a PoseStamped message, and we fill it with the necessary information: the frame_id, the time stamp, and the position and orientation where the table will be placed. Then, we add the table to the scene using the add_box function.

The following is the creation of the grasp object.

In [ ]:
def _add_grasp_block_(self, name):
    p = PoseStamped()
    p.header.frame_id = self._robot.get_planning_frame()
    p.header.stamp = rospy.Time.now()
    p.pose.position.x = 0.62   
    p.pose.position.y = 0.21
    p.pose.position.z = 1.07
    q = quaternion_from_euler(0.0, 0.0, 0.0)
    p.pose.orientation = Quaternion(*q)
    # Grasp Object can size 
    self._scene.add_box(name, p, (0.03, 0.03, 0.09))
    return p.pose

In the above section of the code, what we are doing is the exact same thing as what we did with the table, but for the grasp object. Nothing new here!

After creating the grasp object and the grasp table, we will see how to set the pick position and the place position from the following code snippet. Here, the pose of the grasp object created in the planning scene is retrieved and fed into the place pose in which the Y axis of the place pose is subtracted by 0.06. So, when the pick and place happens, the grasp object will be placed 0.06 meters (6cm) away from the initial pose of the object in the direction of Y.

In [ ]:
# Add table and grap object to the planning scene:
self._pose_table    = self._add_table(self._table_object_name)
self._pose_grasp_obj = self._add_grasp_block_(self._grasp_object_name)
rospy.sleep(1.0)
# Define target place pose:
self._pose_place = Pose()
self._pose_place.position.x = self._pose_grasp_obj.position.x
self._pose_place.position.y = self._pose_grasp_obj.position.y - 0.06
self._pose_place.position.z = self._pose_grasp_obj.position.z
self._pose_place.orientation = Quaternion(*quaternion_from_euler(0.0, 0.0, 0.0))

The next step is to generate the grasp Pose Array data for visualization, and then send the grasp goal to the grasp server. If there is a grasp sequence, it will be published; otherwise, it will show as an error.

In [ ]:
def _generate_grasps(self, pose, width):
    # Create goal:
    goal = GenerateGraspsGoal()
    goal.pose  = pose
    goal.width = width
    ......................
    .......................
    state = self._grasps_ac.send_goal_and_wait(goal)
    if state != GoalStatus.SUCCEEDED:
        rospy.logerr('Grasp goal failed!: %s' % self._grasps_ac.get_goal_status_text())
        return None
    grasps = self._grasps_ac.get_result().grasps
    # Publish grasps (for debugging/visualization purposes):
    self._publish_grasps(grasps)
    return grasps

This function will create a Pose Array data for the pose of the place.

In [ ]:
def _generate_places(self, target):
    # Generate places:
    places = []
    now = rospy.Time.now()
    for angle in numpy.arange(0.0, numpy.deg2rad(360.0), numpy.deg2rad(1.0)):
        # Create place location:
        place = PlaceLocation()
        ............................................
        .......................................
        # Add place:
        places.append(place)
    # Publish places (for debugging/visualization purposes):
    self._publish_places(places)

The next function is **_create_pickup_goal()**, which will create a goal for picking up the grasping object. This goal has to be sent into MoveIt!.

In [ ]:
def _create_pickup_goal(self, group, target, grasps):
    """
    Create a MoveIt! PickupGoal
    """

    # Create goal:
    goal = PickupGoal()

    goal.group_name  = group
    goal.target_name = target

    goal.possible_grasps.extend(grasps)

    goal.allowed_touch_objects.append(target)

    goal.support_surface_name = self._table_object_name

    # Configure goal planning options:
    goal.allowed_planning_time = 7.0

    goal.planning_options.planning_scene_diff.is_diff = True
    goal.planning_options.planning_scene_diff.robot_state.is_diff = True
    goal.planning_options.plan_only = False
    goal.planning_options.replan = True
    goal.planning_options.replan_attempts = 20

    return goal

Also, there is the **_create_place_goal()** function, which creates a place goal for MoveIt.

In [ ]:
def _create_place_goal(self, group, target, places):
    """
    Create a MoveIt! PlaceGoal
    """

    # Create goal:
    goal = PlaceGoal()

    goal.group_name           = group
    goal.attached_object_name = target

    goal.place_locations.extend(places)

    # Configure goal planning options:
    goal.allowed_planning_time = 7.0

    goal.planning_options.planning_scene_diff.is_diff = True
    goal.planning_options.planning_scene_diff.robot_state.is_diff = True
    goal.planning_options.plan_only = False
    goal.planning_options.replan = True
    goal.planning_options.replan_attempts = 20

    return goal

The important functions that are performing picking and placing are given below.
These functions will generate a pick and place sequence, which will be sent to MoveIt and print the results of the motion planning, whether it is successful or not:

In [ ]:
def _pickup(self, group, target, width):
    """
    Pick up a target using the planning group
    """

    # Obtain possible grasps from the grasp generator server:
    grasps = self._generate_grasps(self._pose_coke_can, width)

    # Create and send Pickup goal:
    goal = self._create_pickup_goal(group, target, grasps)

    state = self._pickup_ac.send_goal_and_wait(goal)
    if state != GoalStatus.SUCCEEDED:
        rospy.logerr('Pick up goal failed!: %s' % self._pickup_ac.get_goal_status_text())
        return None

    result = self._pickup_ac.get_result()

    # Check for error:
    err = result.error_code.val
    if err != MoveItErrorCodes.SUCCESS:
        rospy.logwarn('Group %s cannot pick up target %s!: %s' % (group, target, str(moveit_error_dict[err])))

        return False

    return True

In [ ]:
def _place(self, group, target, place):
    """
    Place a target using the planning group
    """

    # Obtain possible places:
    places = self._generate_places(place)

    # Create and send Place goal:
    goal = self._create_place_goal(group, target, places)

    state = self._place_ac.send_goal_and_wait(goal)
    if state != GoalStatus.SUCCEEDED:
        rospy.logerr('Place goal failed!: %s' % self._place_ac.get_goal_status_text())
        return None

    result = self._place_ac.get_result()

    # Check for error:
    err = result.error_code.val
    if err != MoveItErrorCodes.SUCCESS:
        rospy.logwarn('Group %s cannot place target %s!: %s' % (group, target, str(moveit_error_dict[err])))

        return False

    return True

## Grasping in the Real Robot

So, now you've seen how to perform Grasping in MoveIt. But... what about applying this to the real robot? Well, actually, it's quite easy at this point.

<p style="background:#EE9023;color:white;">**Exercise 6.3**</p>
<br>
a) First of all, execute the following commands in order to spawn a table and a Grasping object into the simulation.
<br>
<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">Execute in WebShell #1</p>
</th>
</tr>
</table>

In [ ]:
rosrun gazebo_ros spawn_model -database table -gazebo -model table -x 1.30 -y 0 -z 0

In [ ]:
rosrun gazebo_ros spawn_model -database demo_cube -gazebo -model grasp_cube -x 0.65 -y 0.15 -z 1.07

You should end up with something like this.

<img src="img/add_scene.png" width="600" />

b) Launch MoveIt with Perception.
<br>
<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">Execute in WebShell #1</p>
</th>
</tr>
</table>

In [ ]:
roslaunch fetch_moveit_config myrobot_planning_execution.launch

<img src="img/grasp_perception.png" width="600" />

c) Set the "Planning Attempts" option to something like 6.

<img src="img/planning_att.png" width="300" />

d) Launch the grasping server you created in the previous exercise.
<br>
<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">Execute in WebShell #2</p>
</th>
</tr>
</table>

In [ ]:
roslaunch myrobot_grasping grasp_generator_server.launch

e) Launch this Python script.
<br>
<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">Execute in WebShell #3</p>
</th>
</tr>
</table>

In [ ]:
rosrun myrobot_grasping pick_and_place.py

<p style="background:#AE0202;color:white;">**Expected Result for Exercise 6.3**</p>

RViz view:

<img src="img/grasp_perception2.png" width="600" />

Simulation view:

<img src="img/pick_place.png" width="600" />

<p style="background:#EE9023;color:white;">**End of Exercise 6.3**</p>

And that's all! You've finished the course! Well... almost finished. If you go to the next unit, you'll be able to do a project where you will test everything you've learned during the course.